<a href="https://colab.research.google.com/github/ghthbl/track_your_donation/blob/main/Tracking_individual_donations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

This script is designed to manage and analyze donation allocations for various projects and operational expenses in a charity. The primary functionalities of the script are as follows:

1. **Importing Data**: The script imports CSV files containing information about donations and budgets for different projects and operational expenses. These files are read into pandas DataFrames for easy manipulation and analysis.

2. **Calculating Allocations**: The script calculates the portion of each donation that is allocated to project budgets and operational expenses based on predefined percentages.

3. **Allocating Unlabeled Donations**: Unlabeled donations, which are donations not specifically earmarked for a particular project, are allocated to various project budgets. The script ensures that these donations are distributed appropriately based on the available budget and the size of the donations.

4. **Assigning Donations to Budget Posts**: The script matches donations to specific budget posts within each project. It ensures that the donations are allocated in a way that covers the costs of the budget items.

5. **Handling Operational Expenses**: Donations are also allocated to operational expenses, which are costs that are necessary for the overall functioning of the charity but are not specific to a single project. The script ensures these expenses are appropriately covered by the donations.

6. **Managing the War Chest**: Any remaining donations that are not allocated to project budgets or operational expenses are added to a "war chest," which represents unallocated funds that can be used for future needs.

7. **Generating Reports**: For each donation, the script can generate a detailed report showing how the donation was allocated. This includes the projects, categories, and specific budget items that the donation helped to fund.

### Part 1: Importing libraries and data files<br>
- csv imported into pandas data frames
- operational expenditure ratio calculation

In [67]:
# Imports and basic calculation of op-ex ratio

import pandas as pd
import numpy as np
import re

# Section 1: Import all the CSV files into separate pandas data frames
urls = {
    'donations_unlabeled': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/donations_unlabeled.csv',
    'donations_project_1': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/donations_project_1.csv',
    'donations_project_2': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/donations_project_2.csv',
    'budget_inventory_project_1': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/budget_inventory_project_1.csv',
    'budget_inventory_project_2': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/budget_inventory_project_2.csv',
    'budget_operational_exp': 'https://raw.githubusercontent.com/ghthbl/track_your_donation/main/budget_operational_exp.csv'
}

dataframes = {name: pd.read_csv(url) for name, url in urls.items()}

# Calculate Operational Expense Percentage
total_budget = dataframes['budget_inventory_project_1']['cost'].sum() + dataframes['budget_inventory_project_2']['cost'].sum() + dataframes['budget_operational_exp']['cost'].sum()
operational_expense_percentage = dataframes['budget_operational_exp']['cost'].sum() / total_budget



### Part 2: Assigning donations to relevant project<br>
<br>
- project specific donations go to relevant project<br>
- unlabeled donations used to fill up where needed<br>
- every donation has a unique role in budget<br>
- unused donations go to "War chest"

In [69]:
# Step 1: Prepare Donations DataFrames
for df_name in ['donations_unlabeled', 'donations_project_1', 'donations_project_2']:
    dataframes[df_name] = dataframes[df_name].sort_values(by='donation_in_USD')
    dataframes[df_name]['assign_to_project'] = dataframes[df_name]['donation_in_USD'] * (1 - operational_expense_percentage)
    dataframes[df_name]['assign_to_expenses'] = dataframes[df_name]['donation_in_USD'] * operational_expense_percentage

# Step 2: Allocate Unlabeled Donations to Projects
def allocate_unlabeled_donations(donations_df, budget_df, unlabeled_df):
    donations_df = pd.concat([donations_df, unlabeled_df], ignore_index=True).sort_values(by='donation_in_USD')
    donations_df['cumsum_project'] = donations_df['assign_to_project'].cumsum()
    budget_df['cumsum_cost'] = budget_df['cost'].cumsum()
    allocated_donations = pd.DataFrame()

    for idx, budget_row in budget_df.iterrows():
        budget_post = budget_row['items_package']
        budget_category = budget_row['category']
        budget_cost = budget_row['cost']
        relevant_donations = donations_df[donations_df['cumsum_project'] <= budget_row['cumsum_cost']].copy()
        if not relevant_donations.empty:
            relevant_donations.loc[:, 'budget_post'] = budget_post
            relevant_donations.loc[:, 'category'] = budget_category
            allocated_donations = pd.concat([allocated_donations, relevant_donations], ignore_index=True)
        donations_df = donations_df[donations_df['cumsum_project'] > budget_row['cumsum_cost']]

    return allocated_donations, donations_df

allocated_project_1, remaining_unlabeled = allocate_unlabeled_donations(dataframes['donations_project_1'], dataframes['budget_inventory_project_1'], dataframes['donations_unlabeled'])
allocated_project_2, remaining_unlabeled = allocate_unlabeled_donations(dataframes['donations_project_2'], dataframes['budget_inventory_project_2'], remaining_unlabeled)

# Step 3: Fill Budget Posts with Donations and Operational Expenses
def fill_budget_with_donations(allocated_df, budget_df, project_code, amount_column='assign_to_project'):
    assigned_donations = []
    budget_df = budget_df.sort_values(by='cost')

    for idx, budget_row in budget_df.iterrows():
        budget_post = budget_row['items_package']
        budget_category = budget_row['category']
        budget_cost = budget_row['cost']
        relevant_donations = allocated_df[allocated_df['budget_post'] == budget_post]
        if not relevant_donations.empty:
            tx_id_list = relevant_donations['tx_Id'].tolist()
            assigned_donations.append([project_code, budget_category, budget_post, tx_id_list])

    return pd.DataFrame(assigned_donations, columns=['project_code', 'category', 'budget_post', 'tx_Id_list'])

assigned_project_1 = fill_budget_with_donations(allocated_project_1, dataframes['budget_inventory_project_1'], 'project_1')
assigned_project_2 = fill_budget_with_donations(allocated_project_2, dataframes['budget_inventory_project_2'], 'project_2')

# Process operational expenses separately
def assign_operational_expenses(allocated_project_1, allocated_project_2, operational_df):
    assigned_donations = []
    operational_df = operational_df.sort_values(by='cost', ascending=False)

    relevant_donations_1 = allocated_project_1[allocated_project_1['assign_to_expenses'] > 0]
    relevant_donations_2 = allocated_project_2[allocated_project_2['assign_to_expenses'] > 0]
    relevant_donations = pd.concat([relevant_donations_1, relevant_donations_2]).sort_values(by='assign_to_expenses', ascending=False)

    for _, donation in relevant_donations.iterrows():
        tx_id = donation['tx_Id']
        amount_to_assign = donation['assign_to_expenses']
        assigned_items = []

        for idx, operational_row in operational_df.iterrows():
            if amount_to_assign <= 0:
                break

            operational_post = operational_row['items_package']
            operational_category = operational_row['category']
            operational_cost = operational_row['cost']

            if operational_cost > 0:
                assigned_amount = min(amount_to_assign, operational_cost)
                assigned_items.append([operational_category, operational_post])
                amount_to_assign -= assigned_amount
                operational_df.at[idx, 'cost'] -= assigned_amount

            if len(assigned_items) == 2:
                break

        if assigned_items:
            assigned_donations.append(['op_expenses', tx_id, assigned_items])

    return assigned_donations

# Step 4: Create Assigned Donations DataFrame
assigned_operational_expenses = assign_operational_expenses(allocated_project_1, allocated_project_2, dataframes['budget_operational_exp'])
assigned_operational_expenses_df = pd.DataFrame(assigned_operational_expenses, columns=['project_code', 'tx_Id', 'assigned_items'])
all_assignments = pd.concat([
    assigned_project_1,
    assigned_project_2,
    assigned_operational_expenses_df
], ignore_index=True)

# Step 5: Handle War Chest
remaining_unlabeled['remaining_sum'] = remaining_unlabeled['donation_in_USD']
war_chest = remaining_unlabeled[['tx_Id', 'remaining_sum']]

# Save to CSV files
all_assignments.to_csv('assigned_donations.csv', index=False)
war_chest.to_csv('war_chest.csv', index=False)




### Part 3: Generate individualized report based on tx_Id<br>
- input number from 1 to 750 <br>
- several comma separated numbers accepted (e.g. 2, 5, 708)

In [68]:
# Step 6: Generate donation report

# Function to get donation report
def get_donation_report(tx_id):
    report = f"tx_{tx_id} has been spent on:\n"

    def safe_check(x):
        if isinstance(x, list):
            return f'tx_{tx_id}' in x
        elif isinstance(x, str):
            return f'tx_{tx_id}' in eval(x)
        else:
            return False

    assignments = all_assignments[
        all_assignments['tx_Id_list'].apply(safe_check) |
        (all_assignments['tx_Id'] == f'tx_{tx_id}')
    ]

    if assignments.empty:
        report += "No assignments found for this transaction.\n"
    else:
        report += "Project code | Category | Item\n"
        report += "-" * 40 + "\n"
        for _, row in assignments.iterrows():
            if row['project_code'] == 'op_expenses':
                if isinstance(row['assigned_items'], list):
                    for item in row['assigned_items']:
                        report += f"{row['project_code']} | {item[0]} | {item[1]}\n"
                else:
                    report += f"{row['project_code']} | {row['category']} | {row['budget_post']}\n"
            else:
                report += f"{row['project_code']} | {row['category']} | {row['budget_post']}\n"

    war_chest_amount = war_chest[
        (war_chest['tx_Id'] == f'tx_{tx_id}')
    ]['remaining_sum'].sum()

    original_amount = sum(
        df[(df['tx_Id'] == f'tx_{tx_id}')]['donation_in_USD'].sum()
        for df in [dataframes['donations_unlabeled'], dataframes['donations_project_1'], dataframes['donations_project_2']]
    )

    assigned_amount = original_amount - war_chest_amount
    project_amount = assigned_amount * (1 - operational_expense_percentage)
    operational_amount = assigned_amount * operational_expense_percentage

    report += f"\nTotal sum transferred to war chest: ${war_chest_amount:.2f}\n"
    report += f"Total sum assigned to project: ${project_amount:.2f}\n"
    report += f"Total sum assigned to operational expenses: ${operational_amount:.2f}\n"

    return report

# User input loop
while True:
    user_input = input("Enter tx_Id numbers (comma-separated) or 'q' to quit: ")
    if user_input.lower() == 'q':
        break

    tx_ids = re.findall(r'\d+', user_input)

    if not tx_ids:
        print("No valid tx_Id numbers found. Please try again.")
        continue

    for tx_id in tx_ids:
        print(get_donation_report(tx_id))


Enter tx_Id numbers (comma-separated) or 'q' to quit: 2,250,355,708
tx_2 has been spent on:
Project code | Category | Item
----------------------------------------
project_1 | Tools | item_proj1_4
op_expenses | Repairs | operational_exp_3

Total sum transferred to war chest: $0.00
Total sum assigned to project: $60.00
Total sum assigned to operational expenses: $6.00

tx_250 has been spent on:
Project code | Category | Item
----------------------------------------
project_1 | Tools | item_proj1_3
op_expenses | Recurring payments | operational_exp_6

Total sum transferred to war chest: $0.00
Total sum assigned to project: $46.36
Total sum assigned to operational expenses: $4.64

tx_355 has been spent on:
Project code | Category | Item
----------------------------------------
project_2 | Food/snacks | item_proj2_12
op_expenses | Repairs | operational_exp_5

Total sum transferred to war chest: $0.00
Total sum assigned to project: $125.45
Total sum assigned to operational expenses: $12.55



### More detailed explanation:

1. **Section 1: Import all the CSV files**:
    - This section reads CSV files from specified URLs into pandas DataFrames. These files contain donation data and budget data for different projects and operational expenses.

2. **Section 2: Prepare Donations DataFrames**:
    - This section calculates the amount of each donation that will be allocated to project budgets and operational expenses based on the operational expense percentage.

3. **Section 3: Allocate Unlabeled Donations to Projects**:
    - This function allocates unlabeled donations to project budgets by comparing the cumulative sum of donations with the cumulative sum of budget costs. It ensures that donations are distributed appropriately across different budget posts.

4. **Section 4: Fill Budget Posts with Donations and Operational Expenses**:
    - This function creates a DataFrame showing how donations are assigned to specific budget posts for a given project. It processes each project separately and generates lists of transaction IDs for each budget post.

5. **Section 5: Handle War Chest**:
    - This section calculates the remaining donations that were not allocated to any project or operational expense and assigns them to the war chest. The war chest represents unallocated funds.

6. **Section 6: Function to Generate Donation Report**:
    - This function generates a detailed report for a given transaction ID (`tx_id`). It checks where the donation was allocated, including project budgets and operational expenses, and summarizes the amounts transferred to the war chest, project budgets, and operational expenses.

7. **User Input Loop**:
    - This loop continuously prompts the user for transaction IDs and generates reports for each provided transaction ID.

